In [136]:

import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sb
import plotly.graph_objects as go
from pathlib import Path
import pysam
from tqdm import tqdm
from itertools import repeat
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly
import plotly.io as pio
from random import seed,uniform


In [149]:

def extract_ids_from_coordinates(start1:int,start2:int,end1:int,end2:int,intensity_matrix_df:pd.DataFrame):
    #for start_site in range(start1,start2 + 1):
    #    for end_site in range(end1,end2+1):
    id_collection = []
    temp_intensity_matrix_df = intensity_matrix_df[(intensity_matrix_df["start"] >= start1)]
    temp_intensity_matrix_df = temp_intensity_matrix_df[(temp_intensity_matrix_df["start"] <= start2)]
    temp_intensity_matrix_df = temp_intensity_matrix_df[(temp_intensity_matrix_df["end"] >= end1)]
    temp_intensity_matrix_df = temp_intensity_matrix_df[(temp_intensity_matrix_df["end"] <= end2)]
    print(min([i for i in temp_intensity_matrix_df["start"]]),max([i for i in temp_intensity_matrix_df["end"]]))
    for id_list in temp_intensity_matrix_df["ids"]:
        for id in eval(id_list):
            id_collection.append(id)
    return id_collection




def view_modifications(bamfile_path:str, IVT_path:str, reference_path:str, literature_mod_df_path:str, condition:str, ref_start:int , ref_end:int, id_list:list, fig:go.Figure, colors:list):
    literature_mod_df = pd.read_csv(literature_mod_df_path ,sep="\t",header=None,index_col=None) #"~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed"
    literature_mod_df.columns = ["reference","start","end","modification","A","B","C"]

    psu_mod_df = literature_mod_df.loc[literature_mod_df["modification"] == "psu"]
    Um_mod_df = literature_mod_df.loc[literature_mod_df["modification"] == "Um"]


    fasta_file = pysam.FastaFile(reference_path)
    reference = fasta_file.references[0]
    reference_sequence = str(fasta_file.fetch(reference))

    reads_aligning = [0 for i in range(len(reference_sequence))]
    mod_positions_m6a = [0 for i in range(len(reference_sequence))]
    mod_positions_pseU = [0 for i in range(len(reference_sequence))]
    number_of_basecalled_C = [0 for i in range(len(reference_sequence))]

    bamfile = pysam.AlignmentFile(bamfile_path, mode="rb")
    for i in tqdm(bamfile.fetch(until_eof=True)):
        if not i.query_name in id_list:
            continue
        if i.is_supplementary:
            continue
        start = i.reference_start
        end = i.reference_end
        for index in range(start,end): 
            reads_aligning[index] += 1
        mod_obj = i.modified_bases
        if mod_obj != None:
            # try:
            #     mod_m6a = list(mod_obj[('A', 0, 'a')])
            # except KeyError:
            #     mod_m6a = None
            try:
                mod_pseU = list(mod_obj[('T', 0, 17802)])
            except KeyError:
                mod_pseU = None
            try:
                aligned_pairs = i.get_aligned_pairs(with_seq=True)
                alignment_dict = {}
                for pair_element in aligned_pairs:
                    if None not in pair_element:
                        alignment_dict[str(pair_element[0])] = {"index_query":pair_element[0],"index_reference":pair_element[1],"base_query": str(i.get_forward_sequence())[pair_element[0]],"base_reference": reference_sequence[pair_element[1]]}
                #if mod_m6a != None:
                #    for mod_base in mod_m6a:
                #        p = ((mod_base[1] + 1)/256)
                #        if p >= 0.95 and str(mod_base[0]) in alignment_dict:
                #            mod_positions_m6a[alignment_dict[str(mod_base[0])]["index_reference"]] += 1

                if mod_pseU != None:
                    for mod_base2 in mod_pseU:
                        p = ((mod_base2[1] + 1)/256)
                        if p >= 0.95 and str(mod_base2[0]) in alignment_dict:
                            if ref_start <= alignment_dict[str(mod_base2[0])]["index_reference"] <= ref_end:
                                #print(alignment_dict[str(mod_base2[0])]["index_reference"],alignment_dict[str(mod_base2[0])]["index_query"])
                                mod_positions_pseU[alignment_dict[str(mod_base2[0])]["index_reference"]] += 1
                            
                for pair_element in aligned_pairs:
                    if None not in pair_element:
                        if alignment_dict[str(pair_element[0])]["base_query"] == "C" and alignment_dict[str(pair_element[0])]["base_reference"] == "T":
                            number_of_basecalled_C[pair_element[1]] += 1
            except TypeError:
                print("A type error occured")
    
    reads_aligning = np.array(reads_aligning)
    reads_aligning[reads_aligning == 0] = 1


    positions = [ i + 1 for i in range(len(reads_aligning))]

    mod_positions_m6a = np.array(mod_positions_m6a)
    rel_mod_positions_m6a = mod_positions_m6a/reads_aligning

    mod_positions_pseU = np.array(mod_positions_pseU)
    rel_mod_positions_pseU = mod_positions_pseU/reads_aligning

    number_of_basecalled_C = np.array(number_of_basecalled_C)
    rel_number_of_basecalled_C = number_of_basecalled_C/reads_aligning

    dataset = {
        "position":positions,
        "reads_aligning":reads_aligning,
        "n_pseU":mod_positions_pseU,
        "n_m6a":number_of_basecalled_C,
        "n_C":number_of_basecalled_C,
        "rel_n_pseU":rel_mod_positions_pseU,
        "rel_n_m6a":rel_mod_positions_m6a,
        "rel_n_C":rel_number_of_basecalled_C
    }

    modification_df = pd.DataFrame(dataset)
    modification_df.to_csv(f"{condition}_{ref_start}_{ref_end}_modification_quantification.csv",sep=";",header=True,index=None)

    

        
    
    fig.add_trace(
            go.Scatter(
                x=[index for index in range(len(rel_number_of_basecalled_C))],
                y=[rel_mod_positions_pseU[index]+value for index,value in enumerate(rel_number_of_basecalled_C)],  # Use the same x position and y at the top of the line
                line_color=colors[0],
                showlegend=True,
                name=f"{ref_start}-{ref_end} C/U + pseU freq."
            )
        )
    
    fig.add_trace(
        go.Scatter(
            x=[index for index in range(len(rel_mod_positions_pseU))],
            y=[value for index,value in enumerate(rel_mod_positions_pseU)],  # Use the same x position and y at the top of the line
            line_color=colors[1],
            showlegend=True,
            name=f"{ref_start}-{ref_end} pseU freq."
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x = [i for i in range(0,len(reference_sequence))],
            y = [0 for i in range(0,len(reference_sequence))],
            name= "",
            showlegend=False,
            line_color = "white"
            )
        )
    
    if IVT_path != None:
        IVT_reads_aligning = [0 for i in range(len(reference_sequence))]
        IVT_mod_positions_m6a = [0 for i in range(len(reference_sequence))]
        IVT_mod_positions_pseU = [0 for i in range(len(reference_sequence))]
        IVT_number_of_basecalled_C = [0 for i in range(len(reference_sequence))]
        

        IVT_bamfile = pysam.AlignmentFile(IVT_path, mode="rb")
        counter = 0
        for i in tqdm(IVT_bamfile.fetch(until_eof=True)):
            if counter >= 20000:
                break
            if i.is_supplementary:
                continue
            start = i.reference_start
            end = i.reference_end
            for index in range(start,end):
                IVT_reads_aligning[index] += 1
            mod_obj = i.modified_bases
            if mod_obj != None:
                # try:
                #     mod_m6a = list(mod_obj[('A', 0, 'a')])
                # except KeyError:
                #     mod_m6a = None
                try:
                    mod_pseU = list(mod_obj[('T', 0, 17802)])
                except KeyError:
                    mod_pseU = None
                aligned_pairs = i.get_aligned_pairs(with_seq=True)
                alignment_dict = {}
                for pair_element in aligned_pairs:
                    if None not in pair_element:
                        alignment_dict[str(pair_element[0])] = {"index_query":pair_element[0],"index_reference":pair_element[1],"base_query": str(i.get_forward_sequence())[pair_element[0]],"base_reference": reference_sequence[pair_element[1]]}
                #if mod_m6a != None:
                #    for mod_base in mod_m6a:
                #        p = ((mod_base[1] + 1)/256)
                #        if p >= 0.95 and str(mod_base[0]) in alignment_dict:
                #            IVT_mod_positions_m6a[alignment_dict[str(mod_base[0])]["index_reference"]] += 1

                if mod_pseU != None:
                    for mod_base2 in mod_pseU:
                        p = ((mod_base2[1] + 1)/256)
                        if p >= 0.95 and str(mod_base2[0]) in alignment_dict:
                            IVT_mod_positions_pseU[alignment_dict[str(mod_base2[0])]["index_reference"]] += 1
                            
                for pair_element in aligned_pairs:
                    if None not in pair_element:
                        if alignment_dict[str(pair_element[0])]["base_query"] == "C" and alignment_dict[str(pair_element[0])]["base_reference"] == "T":
                            IVT_number_of_basecalled_C[pair_element[1]] += 1
            counter += 1
        IVT_reads_aligning = np.array(IVT_reads_aligning)
        IVT_reads_aligning[IVT_reads_aligning == 0] = 1


        IVT_positions = [i+1 for i in range(len(IVT_reads_aligning))]

        IVT_mod_positions_m6a = np.array(IVT_mod_positions_m6a)
        IVT_rel_mod_positions_m6a = IVT_mod_positions_m6a/IVT_reads_aligning

        IVT_mod_positions_pseU = np.array(IVT_mod_positions_pseU)
        IVT_rel_mod_positions_pseU = IVT_mod_positions_pseU/IVT_reads_aligning

        IVT_number_of_basecalled_C = np.array(IVT_number_of_basecalled_C)
        IVT_rel_number_of_basecalled_C = IVT_number_of_basecalled_C/IVT_reads_aligning
        
        # IVT_dataset = {
        #     "position":IVT_positions,
        #     "reads_aligning":IVT_reads_aligning,
        #     "n_pseU":IVT_mod_positions_pseU,
        #     "n_m6a":IVT_number_of_basecalled_C,
        #     "n_C":IVT_number_of_basecalled_C,
        #     "rel_n_pseU":IVT_rel_mod_positions_pseU,
        #     "rel_n_m6a":IVT_rel_mod_positions_m6a,
        #     "rel_n_C":IVT_rel_number_of_basecalled_C
        # }

        fig.add_trace(
                go.Scatter(
                    x=[index for index in range(len(IVT_rel_number_of_basecalled_C))],
                    y=[-IVT_rel_mod_positions_pseU[index]-value for index,value in enumerate(IVT_rel_number_of_basecalled_C)],  # Use the same x position and y at the top of the line
                    line_color="rgba(0, 181, 204, 1)",
                    showlegend=True,
                    name="IVT 18S C/U + pseU freq."
                )
            )
        
        fig.add_trace(
                go.Scatter(
                    x=[index for index in range(len(IVT_rel_mod_positions_pseU))],
                    y=[-value for index,value in enumerate(IVT_rel_mod_positions_pseU)],   # Use the same x position and y at the top of the line
                    line_color="rgba(30, 81, 123, 1)",
                    showlegend=True,
                    name="IVT 18S pseU freq."
                )
            )

        for index,T_position in psu_mod_df.iterrows():
            fig.add_shape(
                x0=T_position["end"],
                x1=T_position["end"],
                y0=0,
                y1=1,
                line=dict(
                    color="rgba(90,34,139,0.5)",
                    width=0.3,
                    dash="dash"
                    )
                )
        
        for index,T_position in Um_mod_df.iterrows():
            fig.add_shape(
                x0=T_position["end"],
                x1=T_position["end"],
                y0=0,
                y1=1,
                line=dict(
                    color="rgba(4,59,92,0.5)",
                    width=0.3,
                    dash="dash"
                    )
                )
        
        fig.add_trace(go.Scatter(
            x=[None], y=[None],  # Invisible point, used only for legend entry
            mode='lines',
            line=dict(
                color="rgba(90,34,139,0.7)",
                width=1,
                dash="dash"
            ),
            showlegend=True,
            name="known pseU"  # Legend entry name
        ))
        
        fig.add_trace(go.Scatter(
            x=[None], y=[None],  # Invisible point, used only for legend entry
            mode='lines',
            line=dict(
                color="rgba(4,59,92,0.7)",
                width=1,
                dash="dash"
            ),
            showlegend=True,
            name="known Um"  # Legend entry name
        ))
    fig.update_layout(
        title=f"Modification basecalling",
        xaxis=dict(title="Position on reference",gridcolor = "white",tickformat="d"),
        yaxis=dict(title="Modification frequency",gridcolor = "white"),
        plot_bgcolor='rgba(0,0,0,0)'
    )
    #fig.write_image(f"{condition}_positions_{start}_{end}_general_modification_ratio_with_Um.svg",format="svg")
    
    #Coverage window 
    # fig.update_layout(
    #     xaxis=dict(range=[max(start-10,0),min(end+10,13551)])
    # )
    # fig.write_image(f"{condition}_positions_{start}_{end}_modification_ratio_with_Um.svg",format="svg")
    # fig.show()
    

In [ ]:
IVPA_Nuc_intensity_matrix_df = pd.read_csv("~/Synology/Data_nano_ribolyzer/directRNA_004/20231114_RNA004_IVPA_Nuc/intensity_matrix/intensity_matrix.csv",sep=";",header=0)
IVPA_Nuc_bamfile = "~/Synology/Data_nano_ribolyzer/directRNA_004/20231114_RNA004_IVPA_Nuc/filtered_pod5/filtered_pod5_rebasecalled_psU_m6A_aligned.bam"        
IVPA_Nuc_439_448_6377_6381_id_list = extract_ids_from_coordinates(439,448,6377,6381,IVPA_Nuc_intensity_matrix_df)   
print(len(IVPA_Nuc_439_448_6377_6381_id_list),IVPA_Nuc_439_448_6377_6381_id_list[0])

IVPA_Nuc_3656_3670_5517_5524_id_list = extract_ids_from_coordinates(3656,3670,5517,5524,IVPA_Nuc_intensity_matrix_df)
print(len(IVPA_Nuc_3656_3670_5517_5524_id_list),IVPA_Nuc_3656_3670_5517_5524_id_list[0])

IVPA_Nuc_3656_3670_6379_6381_id_list = extract_ids_from_coordinates(3656,3670,6379,6381,IVPA_Nuc_intensity_matrix_df)
print(len(IVPA_Nuc_3656_3670_6379_6381_id_list),IVPA_Nuc_3656_3670_6379_6381_id_list[0])


In [ ]:
layout = go.Layout(height = 800)
fig1 = go.Figure(layout=layout)

view_modifications(IVPA_Nuc_bamfile,
                   IVT_path=None,
                   reference_path="~/wf-nanoribolyzer/references/RNA45SN1.fasta",
                   literature_mod_df_path="~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed",
                   condition = "IVPA_Nuc",
                   ref_start=439,
                   ref_end=6381,
                   id_list=IVPA_Nuc_439_448_6377_6381_id_list,
                   fig=fig1,
                   colors = ["rgba(118,0,0,0.4)","rgba(118,0,0,0.2)"]
                )
view_modifications(IVPA_Nuc_bamfile,
                   IVT_path = None,
                   reference_path="~/wf-nanoribolyzer/references/RNA45SN1.fasta",
                   literature_mod_df_path="~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed",
                   condition = "IVPA_Nuc",
                   ref_start=3656,
                   ref_end=5524,
                   id_list=IVPA_Nuc_3656_3670_5517_5524_id_list,
                   fig=fig1,
                   colors = ["rgba(0,118,0,0.4)","rgba(0,118,0,0.2)"]
                )

view_modifications(IVPA_Nuc_bamfile,
                   IVT_path = "~/Synology/Data_nano_ribolyzer/directRNA_004/IVT_18S/filtered_pod5/filtered_pod5_basecalled.bam",
                   reference_path="~/wf-nanoribolyzer/references/RNA45SN1.fasta",
                   literature_mod_df_path="~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed",
                   condition = "IVPA_Nuc",
                   ref_start=365,
                   ref_end=6381,
                   id_list=IVPA_Nuc_3656_3670_6379_6381_id_list,
                   fig=fig1,
                   colors = ["rgba(191,85,236,0.4)","rgba(191,85,236,0.2)"]
                )
fig1.show()
fig1.write_image("IVPA_Nucleus_pseU_freq_extracted_18S_intermediates.svg",format="svg")

In [ ]:
NP_Nuc_intensity_matrix_df = pd.read_csv("~/Synology/Data_nano_ribolyzer/directRNA_004/20231114_RNA004_NP_Nuc/intensity_matrix/intensity_matrix.csv",sep=";",header=0)
NP_Nuc_bamfile = "~/Synology/Data_nano_ribolyzer/directRNA_004/20231114_RNA004_NP_Nuc/filtered_pod5/filtered_pod5_rebasecalled_psU_m6A_aligned.bam"        
NP_Nuc_439_448_6377_6381_id_list = extract_ids_from_coordinates(439,448,6377,6381,NP_Nuc_intensity_matrix_df)   
print(len(NP_Nuc_439_448_6377_6381_id_list),NP_Nuc_439_448_6377_6381_id_list[0])

NP_Nuc_3656_3670_5517_5524_id_list = extract_ids_from_coordinates(3656,3670,5517,5524,NP_Nuc_intensity_matrix_df)
print(len(NP_Nuc_3656_3670_5517_5524_id_list),NP_Nuc_3656_3670_5517_5524_id_list[0])

NP_Nuc_3656_3670_6379_6381_id_list = extract_ids_from_coordinates(3656,3670,6379,6381,NP_Nuc_intensity_matrix_df)
print(len(NP_Nuc_3656_3670_6379_6381_id_list),NP_Nuc_3656_3670_6379_6381_id_list[0])

In [ ]:
layout = go.Layout(height = 800)
fig2 = go.Figure(layout=layout)

view_modifications(NP_Nuc_bamfile,
                   IVT_path=None,
                   reference_path="~/wf-nanoribolyzer/references/RNA45SN1.fasta",
                   literature_mod_df_path="~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed",
                   condition = "NP_Nuc",
                   ref_start=439,
                   ref_end=6381,
                   id_list=IVPA_Nuc_439_448_6377_6381_id_list,
                   fig=fig1,
                   colors = ["rgba(118,0,0,0.4)","rgba(118,0,0,0.2)"]
                )
view_modifications(NP_Nuc_bamfile,
                   IVT_path = None,
                   reference_path="~/wf-nanoribolyzer/references/RNA45SN1.fasta",
                   literature_mod_df_path="~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed",
                   condition = "NP_Nuc",
                   ref_start=3656,
                   ref_end=5524,
                   id_list=NP_Nuc_3656_3670_5517_5524_id_list,
                   fig=fig1,
                   colors = ["rgba(0,118,0,0.4)","rgba(0,118,0,0.2)"]
                )

view_modifications(NP_Nuc_bamfile,
                   IVT_path = "~/Synology/Data_nano_ribolyzer/directRNA_004/IVT_18S/filtered_pod5/filtered_pod5_basecalled.bam",
                   reference_path="~/wf-nanoribolyzer/references/RNA45SN1.fasta",
                   literature_mod_df_path="~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed",
                   condition = "NP_Nuc",
                   ref_start=365,
                   ref_end=6381,
                   id_list=NP_Nuc_3656_3670_6379_6381_id_list,
                   fig=fig1,
                   colors = ["rgba(191,85,236,0.4)","rgba(191,85,236,0.2)"]
                )
fig2.show()
fig2.write_image("IVPA_Nucleus_pseU_freq_extracted_18S_intermediates.svg",format="svg")

In [ ]:
IVPA_Cyt_intensity_matrix_df = pd.read_csv("~/Synology/Data_nano_ribolyzer/directRNA_004/20231114_RNA004_IVPA_Cyt/intensity_matrix/intensity_matrix.csv",sep=";",header=0)
IVPA_Cyt_bamfile = "~/Synology/Data_nano_ribolyzer/directRNA_004/20231114_RNA004_IVPA_Cyt/filtered_pod5/filtered_pod5_rebasecalled_psU_m6A_aligned.bam"        
IVPA_Cyt_439_448_6377_6381_id_list = extract_ids_from_coordinates(439,448,6377,6381,IVPA_Cyt_intensity_matrix_df)   
print(len(IVPA_Cyt_439_448_6377_6381_id_list),IVPA_Cyt_439_448_6377_6381_id_list[0])

IVPA_Cyt_3656_3670_5517_5524_id_list = extract_ids_from_coordinates(3656,3670,5517,5524, IVPA_Cyt_intensity_matrix_df)
print(len(IVPA_Cyt_3656_3670_5517_5524_id_list),IVPA_Cyt_3656_3670_5517_5524_id_list[0])

IVPA_Cyt_3656_3670_6379_6381_id_list = extract_ids_from_coordinates(3656,3670,6379,6381,IVPA_Cyt_intensity_matrix_df)
print(len(IVPA_Cyt_3656_3670_6379_6381_id_list),IVPA_Cyt_3656_3670_6379_6381_id_list[0])

In [ ]:
layout = go.Layout(height = 800)
fig3 = go.Figure(layout=layout)

view_modifications(IVPA_Cyt_bamfile,
                   reference_path="~/wf-nanoribolyzer/references/RNA45SN1.fasta",
                   literature_mod_df_path="~/wf-nanoribolyzer/references/rRNA_modifications_conv.bed",
                   condition = "IVPA_Cyt",
                   ref_start=3656,
                   ref_end=5524,
                   id_list=IVPA_Cyt_3656_3670_5517_5524_id_list,
                   fig=fig1,
                   colors = ["rgba(118,0,0,0.4)","rgba(118,0,0,0.2)"]
                )
fig3.show()